<a href="https://colab.research.google.com/github/amedeotolace/Amedeo_Thesis_2/blob/main/IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install tensorflow tensorflow_decision_forests
!pip install datapackage
!pip install pandas
!pip install keras

In [ ]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/machine-learning/kddcup99/datapackage.json'
names = [
    'count',
    'diff_srv_rate',
    'dst_bytes',
    'dst_host_count',
    'dst_host_diff_srv_rate',
    'dst_host_rerror_rate',
    'dst_host_same_src_port_rate',
    'dst_host_same_srv_rate',
    'dst_host_serror_rate',
    'dst_host_srv_count',
    'dst_host_srv_diff_host_rate',
    'dst_host_srv_rerror_rate',
    'dst_host_srv_serror_rate',
    'duration',
    'flag',
    'hot',
    'is_guest_login',
    'is_hot_login',
    'label',
    'land',
    'logged_in',
    'num_access_files',
    'num_compromised',
    'num_failed_logins',
    'num_file_creations',
    'num_outbound_cmds',
    'num_root',
    'num_shells',
    'protocol_type',
    'rerror_rate',
    'root_shell',
    'same_srv_rate',
    'serror_rate',
    'service',
    'src_bytes',
    'srv_count',
    'srv_diff_host_rate',
    'srv_rerror_rate',
    'srv_serror_rate',
    'su_attempted',
    'urgent',
    'wrong_fragment',
]

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        dataset_df = pd.read_csv(resource.descriptor['path'], header=None, names=names)
        print (dataset_df)

In [ ]:
#Split del dataset in train & test

import numpy as np

def split_dataset(dataset, test_ratio=0.30):
  """Divide un pandas dataframe, in questo caso suddiviamo il dataset caricato in 2"""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)

#Lunghezza dei dataset di test e di train
"""print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))
"""
print(test_ds_pd.T)

In [ ]:
#Random Forest

import tensorflow_decision_forests as tfdf
import pandas as pd

#Convertiamo da Pandas dataframe ad un dataset Tensorflow
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='protocol_type')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='protocol_type')

# Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)

# Export the model to a SavedModel.
model.save("project/model")

In [ ]:
!pip install tensorflow numpy scikit-learn


In [ ]:
print(dataset_df)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense

# Convert categorical features to one-hot encoding
df = pd.get_dummies(dataset_df, columns=["protocol_type", "service", "flag"])
print(df)

In [ ]:
# Separate features and labels
X = df.drop("label", axis=1)
y = df["label"]
print(X)
print(y)

In [ ]:
# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the non-symmetric deep autoencoder model
input_layer = Input(shape=(X.shape[1],))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(X.shape[1], activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Encode the data
encoder = Model(input_layer, encoded)
encoded_train = encoder.predict(X_train)
encoded_test = encoder.predict(X_test)

# Use the encoded data for classification (for example, with a simple classifier)
classifier_input = Input(shape=(32,))
classifier_output = Dense(1, activation='sigmoid')(classifier_input)
classifier = Model(classifier_input, classifier_output)

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the classifier
classifier.fit(encoded_train, y_train, epochs=10, batch_size=32, shuffle=True, validation_data=(encoded_test, y_test))

# Evaluate the classifier on the test data
y_pred = classifier.predict(encoded_test)
y_pred_binary = np.round(y_pred)
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy}")